# Import Libraries

In [1]:
import pandas as pd
import os
import requests
import psycopg2
import psycopg2.extras
import json
import datanews
import time

C:\Users\imaia\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.3) or chardet (4.0.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [44]:
# Datanews API Credentials

api_key = "0fdlbukyla9uenffppsig1faq"


In [109]:
# Stocks we will collect data on 
tickers= ["AAPL", "FB", "ADBE", "INTC", "PYPL", "BABA", "QCOM", "SHOP", "AMD", "MSFT"]

# Dates we will collect news for each stock
from_dates = ["2020-04-01", "2020-05-01","2020-06-01","2020-07-01","2020-08-01","2020-09-01","2020-10-01",
             "2020-11-01","2020-12-01","2021-01-01","2021-02-01","2021-03-01"]

to_dates = ["2020-05-01","2020-06-01","2020-07-01","2020-08-01","2020-09-01","2020-10-01","2020-11-01",
            "2020-12-01","2021-01-01","2021-02-01","2021-03-01", "2021-04-01"]

In [67]:
# Retreive news for each stock for the past year in increments of months

news_requests = []

for ticker in tickers:
    for i in range(len(to_dates)):
        params = {'apiKey': api_key,
                  'q': ticker,
                  'from': from_dates[i],
                  'to': to_dates[i],
                  'language': 'en',
                  'size': 100}

        r = requests.get("http://api.datanews.io/v1/news", params=params)

        news_requests.append(r)
        
        time.sleep(3)



In [63]:
print((news_requests[0].json()['hits']))

[{'url': 'https://www.ccn.com/dow-futures-soar-as-new-york-begins-winning-the-coronavirus-war/', 'source': 'ccn.com', 'authors': [], 'title': 'Dow Futures Soar as New York Begins Winning the Coronavirus War', 'pubDate': '2020-04-06T05:41:20+00:00', 'country': 'us', 'language': 'en', 'description': 'Dow futures surged 700 points as stock market bulls finally get some positive news about the coronavirus as N.Y. data shows a peak is near.', 'imageUrl': 'https://www.ccn.com/wp-content/uploads/2020/04/New-York-NTB-Coronavirus.jpg', 'content': '\n• Dow Futures soared overnight as data in Europe and New York suggests the peak of the coronavirus outbreak is now within reach.\n• Governor Andrew Cuomo also noted a huge portion of cases have been de-hospitalized.\n• The Dow Jones is not in the clea ... [+4212 chars]'}]


In [100]:
# Store news in a dataframe

news_data = pd.DataFrame()

for i in news_requests:
    
    if news_data.empty == True:
        news_data = pd.DataFrame(i.json()['hits'])
    else:
        try:
            mn = pd.DataFrame(i.json()['hits'])
        except:
            pass
        
        news_data = news_data.append(mn)

In [101]:
news_data.shape

(8593, 10)

In [102]:
# remove any rows with null values
news_data = news_data.dropna(how='any',axis=0) 
news_data.shape

(8267, 10)

In [103]:
news_data.head()

,url,source,authors,title,pubDate,country,language,description,imageUrl,content
0,https://www.ccn.com/dow-futures-soar-as-new-yo...,ccn.com,[],Dow Futures Soar as New York Begins Winning th...,2020-04-06T05:41:20+00:00,us,en,Dow futures surged 700 points as stock market ...,https://www.ccn.com/wp-content/uploads/2020/04...,\n• Dow Futures soared overnight as data in Eu...
0,https://www.smarteranalyst.com/yahoo/apple-sna...,smarteranalyst.com,[Harriet Lefton],"Apple Snaps Up AI Startup Inductiv, As Analyst...",2020-05-28T02:38:10+00:00,,en,Apple (AAPL) has snapped up Canadian machine l...,https://www.smarteranalyst.com/wp-content/uplo...,Apple (AAPL) has snapped up Canadian machine l...
1,https://www.smarteranalyst.com/yahoo/gates-fou...,smarteranalyst.com,[Sharon Wrobel],"Gates Foundation Buys Up Amazon, Apple, Twitte...",2020-05-25T11:02:06+00:00,,en,The Bill & Melinda Gates Foundation Trust has ...,https://www.smarteranalyst.com/wp-content/uplo...,The Bill & Melinda Gates Foundation Trust has ...
2,https://www.smarteranalyst.com/yahoo/apple-chi...,smarteranalyst.com,[Sharon Wrobel],"Apple China Sales On Recovery Path In April, i...",2020-05-25T03:13:23+00:00,,en,Apple Inc.’s (AAPL) China sales continued to r...,https://www.smarteranalyst.com/wp-content/uplo...,Apple Inc.’s (AAPL) China sales continued to r...
3,http://www.smarteranalyst.com/uncategorized/in...,smarteranalyst.com,[Sharon Wrobel],"Intel, Taiwan Semiconductor Said to Be in Talk...",2020-05-13T05:13:13+00:00,,en,The Trump administration is holding discussion...,https://www.smarteranalyst.com/wp-content/uplo...,The Trump administration is holding discussion...


In [105]:
# Replace all comas with spaces, since exporting to csv was generating wrong with comas included
news_data['url'] = news_data['url'].replace(',','', regex=True)
news_data['source'] = news_data['source'].replace(',','', regex=True)
news_data['title'] = news_data['title'].replace(',','', regex=True)


In [106]:
news_data['title'].iloc[107]

'Bring Your "A" Game: Amazon Apple Alphabet (And FB) Get Ready To Report After Close'

In [107]:
# Store stock news data in PostgreSQL database

conn = None

try:
    conn = psycopg2.connect(host="localhost", database="postgres", port=5432, user="postgres", password="Aki12345+")

    command = "INSERT INTO stock_news (headline, source, url, date) \
    VALUES (%s, %s, %s, %s);"

    cur = conn.cursor()
    for index, row in news_data.iterrows():
        cur.execute(command, (str(row['title']),str(row['source']),
                              str(row['url']), str(row['pubDate'])))

except (Exception, psycopg2.DatabaseError):
    print("Could not connect to database")

finally:
    if conn is not None:
        conn.commit()
        conn.close()
        cur.close()